<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/Catboost_20240829.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.0 MB/s eta 0:00:00


In [4]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [5]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [19]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_trainV8.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_testV8.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [7]:
df_train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren,HasChildren,IncomePerChild,AnnualIncomePerTrip,PitchPlusFollowups,AdultTravelers
0,3489,48.0,1,2,13.0,1,0,1.0,4.0,1,3.0,7.0,0,3,1.000000,496950.0,NaN,2,1,0.0,0,4.969500e+10,8.519131e+05,21.0,1.0
1,3490,30.0,1,2,12.0,1,1,1.0,4.0,3,3.0,4.0,1,3,3.445506,300000.0,NaN,2,0,0.0,0,3.000000e+10,8.999978e+05,20.0,1.0
2,3491,25.0,1,1,9.0,0,1,1.0,4.0,4,3.0,1.0,0,3,4.000000,260000.0,NaN,3,0,0.0,0,2.600000e+10,3.119969e+06,17.0,1.0
3,3492,21.0,0,2,7.0,0,0,1.0,4.0,4,4.0,1.0,0,3,4.077367,259875.0,NaN,3,1,0.0,0,2.598750e+10,3.118469e+06,15.0,1.0
4,3493,41.0,0,1,7.0,0,0,1.0,4.0,4,3.0,1.0,0,4,4.000000,268830.0,NaN,0,1,0.0,0,2.688300e+10,3.225928e+06,15.0,1.0


In [20]:
# Config cell
target = "ProdTaken"

In [21]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['HasChildren','IncomePerChild','AnnualIncomePerTrip','PitchPlusFollowups','AdultTravelers']
 #['IncomePerAge','FamilySize','IncomePerFamilySize','IncomePerTrips','PersonTripsProduct','IncomePerChild','LogMonthlyIncome','PitchFollowupInteraction','TripsPerAge',
#'IncomeStarInteraction','FollowupEfficiency','IncomePerPerson','StarPerCityTier','IncomePerStar','LuxuryAffinity','FollowupPerTrip']
numerical_features = default_numerical_features + added_numerical_features



#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=['id',target,'HasChildren']
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar', 'HasChildren', 'IncomePerChild', 'AnnualIncomePerTrip', 'PitchPlusFollowups', 'AdultTravelers']
categorical_features ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar', 'Passport', 'PitchSatisfactionScore', 'Designation', 'ProdTaken', 'MaritalStatus']
features for training:['Age', 'TypeofContact', 'CityTier', 'DurationOfPitch', 'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'Designation', 'MonthlyIncome', 'MaritalStatus', 'OwnCar', 'NumberOfChildren', 'IncomePerChild', 'AnnualIncomePerTrip', 'PitchPlusFollowups', 'AdultTravelers']


In [22]:
cols_category=categorical_features

In [23]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [24]:
train_y.head()

,ProdTaken
0,1.0
1,0.0
2,1.0
3,0.0
4,1.0


In [25]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    'depth':1,
    'l2_leaf_reg' : 8,
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    'one_hot_max_size' : 25,
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [26]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [27]:
!pip install shap

In [28]:
import shap

In [30]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []
shap_values_list = []

list_feature_importances = []  # 特徴量重要度を格納するリスト

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x[features].iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x[features].iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

    # 特徴量の重要度を追加
    feature_importances = model_cat.get_feature_importance()
    list_feature_importances.append(feature_importances)

# AUCの出力
print("Mean AUC:", np.mean(list_metrics_auc))
print("Mean F1 Score:", np.mean(list_metrics_f1))
print("Median Cutoff:", np.median(list_cutoff))

# 特徴量重要度の集計
mean_feature_importances = np.mean(list_feature_importances, axis=0)

# 結果の表示
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Mean Importance': mean_feature_importances
}).sort_values(by='Mean Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

0:	test: 0.5987561	best: 0.5987561 (0)	total: 2.88ms	remaining: 2.87s
100:	test: 0.8524253	best: 0.8524253 (100)	total: 576ms	remaining: 5.13s
200:	test: 0.8565881	best: 0.8570324 (159)	total: 1.14s	remaining: 4.54s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8570323812
bestIteration = 159

Shrink model to first 160 iterations.
Fold: 1, AUC: 0.8570323812031064, f1 score: 0.7222778779581023 Threshold: 0.21139103976983742
0:	test: 0.6954719	best: 0.6954719 (0)	total: 15.3ms	remaining: 15.2s
100:	test: 0.8242398	best: 0.8242398 (100)	total: 825ms	remaining: 7.35s
200:	test: 0.8339805	best: 0.8339805 (200)	total: 1.74s	remaining: 6.9s
300:	test: 0.8389167	best: 0.8396077 (291)	total: 2.29s	remaining: 5.33s
400:	test: 0.8422404	best: 0.8422404 (400)	total: 2.8s	remaining: 4.18s
500:	test: 0.8432605	best: 0.8440832 (487)	total: 3.22s	remaining: 3.21s
600:	test: 0.8436225	best: 0.8442148 (550)	total: 3.69s	remaining: 2.45s
700:	test: 0.8440832	best: 0.8443794 (633)	to

In [ ]:
0.8497045879206161

SyntaxError: invalid syntax (<ipython-input-81-76ea1f53a6c0>, line 1)

In [ ]:
0.8437365146029495

In [31]:
import pickle

In [32]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v12{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [33]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x7d2848b19630>, Proba: [0.01355773 0.02475626 0.0360633  ... 0.07731399 0.03559452 0.00184477]
Model: <catboost.core.CatBoostClassifier object at 0x7d2849582e60>, Proba: [0.02909037 0.06344698 0.07571399 ... 0.16205036 0.06661662 0.00258255]
Model: <catboost.core.CatBoostClassifier object at 0x7d2848ae6a10>, Proba: [0.03879153 0.08964563 0.11112691 ... 0.24537928 0.10815506 0.00555456]
Model: <catboost.core.CatBoostClassifier object at 0x7d2848b382e0>, Proba: [0.051437   0.13036282 0.14575474 ... 0.3396673  0.14022192 0.00629054]
Model: <catboost.core.CatBoostClassifier object at 0x7d2848ae74c0>, Proba: [0.06275489 0.16689703 0.18581478 ... 0.43209416 0.17269807 0.00747531]
Model: <catboost.core.CatBoostClassifier object at 0x7d2848b3bf40>, Proba: [0.07437803 0.1984235  0.22154697 ... 0.5132422  0.20572464 0.00888703]
Model: <catboost.core.CatBoostClassifier object at 0x7d28808473d0>, Proba: [0.08751643 0.23724194 0.25783221 ... 0.611

In [34]:
count = 70

In [35]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)